# Task 1.
For each regular expression `rexpr` and the corresponding string `s` in the following table, indicate the span of string `s` that is matched by `rexpr` (i.e., the output of `re.search(rexpr, s)` in Python).

| rexpr | s        |
|-------|----------|
| ha*   | hahaha   |
| ha*   | haaa     |
| ha*   | hihahaha |
| (ha)* | hahaha   |
| (ha)* | haaa     |
| (ha)* | hihahaha |


In [23]:
import pandas as pd
import re

rexprs = ['ha*'] * 3 + ['(ha)*'] * 3
ss = ['hahaha', 'haaa', 'hihahaha', 'hahaha', 'haaa', 'hihahaha']

table = pd.DataFrame({'rexpr': rexprs, 's': ss})

for index, row in table.iterrows():
    rexpr = row['rexpr']
    s = row['s']
    match = re.search(rexpr, s)
    if match:
        print(match)
        # print(f"Match found for rexpr '{rexpr}' in string '{s}' at span {match.span()}")
    else:
        print(f"No match found for rexpr '{rexpr}' in string '{s}'")
        

<re.Match object; span=(0, 2), match='ha'>
<re.Match object; span=(0, 4), match='haaa'>
<re.Match object; span=(0, 1), match='h'>
<re.Match object; span=(0, 6), match='hahaha'>
<re.Match object; span=(0, 2), match='ha'>
<re.Match object; span=(0, 0), match=''>


# Task 2.
In Porter stemming algorithm, the matching condition of one of the rewrite rules is that
the word to be matched contains a vowel (a, e, i, o, u) before ending with ing. The intent
is that when a word satisfies this condition, it is a verb in continuous tense, and so its ing
ending can be removed to convert the verb into its base form.

Give a regular expression that will match a word that satisfies the above-mentioned
condition. Assume that the string to be matched is a word consisting of lowercase letters
and digits.

### Conditions
if word includes vowel, and ends with "ing".

In [18]:
word='hello'

def porter_stemming_algorithm(word):
    """
    NOTE: ### Documentation written partaly using GitHub Copilot for added clarity!!! ### 
    
    Applies the Porter Stemming Algorithm to the given word.

    Parameters:
    word (str): The word to be stemmed.

    Returns:
    None

    This function checks if the word ends with 'ing' and contains at least one vowel. If both conditions are met, it is assumed to be a verb in continuous tense. If only the second condition is met, it is assumed to be a regular verb. If only the first condition is met, it is assumed to be a word with a vowel.

    Examples:
    >>> porter_stemming_algorithm('running')
    passes first and second condition, probably a verb in continuous tense

    >>> porter_stemming_algorithm('run')
    passes second condition only

    >>> porter_stemming_algorithm('apple')
    passes first condition only
    """
    if any(vowel in word for vowel in ['a', 'e', 'i', 'o', 'u']) and word.endswith('ing'):
        print('passes first and second condition, probably a verib in continious tense')
    elif any(vowel in word for vowel in ['a', 'e', 'i', 'o', 'u']):
        print('passes first condition only')
    elif word.endswith('ing'):
        print('passes second condition only')

porter_stemming_algorithm(word=word)


passes first condition only


# Task 3.
### 3. Let the training text be:
`tin singing sterling jest lingo singing jest singing tin`

Give a trace of the byte-pair encoding (BPE) algorithm given the above training text, where the number of merges `k = 4`. Show the merge operation and the resulting vocabulary in each step of the algorithm. If there is a tie in the choice of merge operations, a merge operation that results in a merged token with the smallest number of characters is preferred.

For each of the following strings, show the tokenized string by applying the token segmenter learned:

(a) `ingest`  
(b) `sting`  
(c) `interest`


### Q: What is BPE, and how does one implement it?
[GeeksForGeeks](https://www.geeksforgeeks.org/byte-pair-encoding-bpe-in-nlp/)
1. Initialize the vocabulary with all the bytes or characters in the text corpus
2. Calculate the frequency of each byte or character in the text corpus.
3. Repeat the following steps until the desired vocabulary size is reached:
    1. Find the most frequent pair of consecutive bytes or characters in the text corpus
    2. Merge the pair to create a new subword unit.
    3. Update the frequency counts of all the bytes or characters that contain the merged pair.
4. Add the new subword unit to the vocabulary.

In [19]:
import re
from collections import defaultdict
from typing import Dict, Tuple


class BytePairEncoding:
    """
    Implements the Byte Pair Encoding (BPE) algorithm for tokenizing text by iteratively merging the most frequent pairs of symbols.
    """

    def __init__(self, corpus: str):
        """
        Initializes the BPE model with a corpus.

        :param corpus: A string containing the training text for BPE.
        """
        self.corpus = corpus
        self._vocab = self._get_initial_vocab()
        self.merges = []  # To store the order of merges

    def _get_initial_vocab(self) -> Dict[str, int]:
        """
        Builds the initial vocabulary from the corpus. Each word in the corpus is split into characters, and each character
        sequence is considered as an initial token.

        :return: A dictionary where keys are tokenized words and values are their frequencies in the corpus.
        """
        vocab = defaultdict(int)
        for word in self.corpus.split():
            # Each character in the word is separated by a space to form the initial vocabulary
            vocab[" ".join(list(word))] += 1
        return vocab

    def _get_stats(self) -> Dict[Tuple[str, str], int]:
        """
        Calculates the frequency of each pair of consecutive symbols in the vocabulary.

        :return: A dictionary where keys are pairs of symbols and values are their frequencies.
        """
        pairs = defaultdict(int)
        for word, freq in self._vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                # Count the frequency of each adjacent pair of symbols
                pairs[symbols[i], symbols[i + 1]] += freq
        return pairs

    def _merge_vocab(self, pair: Tuple[str, str]) -> None:
        """
        Merges the most frequent pair of symbols in the vocabulary.

        :param pair: A tuple representing the pair of symbols to be merged.
        """
        v_out = {}
        bigram = re.escape(" ".join(pair))
        p = re.compile(r"(?<!\S)" + bigram + r"(?!\S)")

        for word in self._vocab:
            # Replace the most frequent pair with its merged form in the vocabulary
            w_out = p.sub("".join(pair), word)
            v_out[w_out] = self._vocab[word]

        self._vocab = v_out
        self.merges.append(pair)  # Store the merge operation

    def train(self, n: int) -> None:
        """
        Performs the BPE algorithm for a specified number of merges.

        :param n: The number of merge operations to perform.
        """
        print(f"Initial Vocabulary: {self._vocab}\n")

        for i in range(n):
            pairs = self._get_stats()
            if not pairs:
                break  # No more pairs to merge

            # Find the most frequent pair and handle ties
            max_freq = max(pairs.values())
            best_pairs = [pair for pair in pairs if pairs[pair] == max_freq]

            # Tie-breaking: choose the pair that results in the smallest merged token
            best = min(best_pairs, key=lambda x: len("".join(x)))

            self._merge_vocab(best)

            print(f"Step {i + 1}: Merge {best} -> Vocabulary: {self._vocab}\n")

    def tokenize(self, word: str) -> str:
        """
        Tokenizes a new word using the learned BPE merges.

        :param word: The word to tokenize.
        :return: The tokenized version of the word.
        """
        word = (
            " ".join(list(word)) + " </w>"
        )  # Initialize the word with spaces between characters
        for merge in self.merges:
            bigram = " ".join(merge)
            word = re.sub(r"\b" + bigram + r"\b", "".join(merge), word)
        return word.replace("</w>", "")

    @property
    def vocab(self) -> Dict[str, int]:
        """
        Returns the current vocabulary after training.

        This property ensures that the vocabulary is accessed in a read-only manner.

        :return: The vocabulary dictionary after BPE merges.
        """
        return self._vocab

In [20]:
corpus = "tin singing sterling jest lingo singing jest singing tin"
n = 4

bpe = BytePairEncoding(corpus)
bpe.train(n)
final_vocab = bpe.vocab

print(f"Final Vocabulary: {final_vocab}")

Initial Vocabulary: defaultdict(<class 'int'>, {'t i n': 2, 's i n g i n g': 3, 's t e r l i n g': 1, 'j e s t': 2, 'l i n g o': 1})

Step 1: Merge ('i', 'n') -> Vocabulary: {'t in': 2, 's in g in g': 3, 's t e r l in g': 1, 'j e s t': 2, 'l in g o': 1}

Step 2: Merge ('in', 'g') -> Vocabulary: {'t in': 2, 's ing ing': 3, 's t e r l ing': 1, 'j e s t': 2, 'l ing o': 1}

Step 3: Merge ('s', 't') -> Vocabulary: {'t in': 2, 's ing ing': 3, 'st e r l ing': 1, 'j e st': 2, 'l ing o': 1}

Step 4: Merge ('s', 'ing') -> Vocabulary: {'t in': 2, 'sing ing': 3, 'st e r l ing': 1, 'j e st': 2, 'l ing o': 1}

Final Vocabulary: {'t in': 2, 'sing ing': 3, 'st e r l ing': 1, 'j e st': 2, 'l ing o': 1}


In [21]:
tokenized_a = bpe.tokenize("ingest")
tokenized_b = bpe.tokenize("sting")
tokenized_c = bpe.tokenize("interest")

print(f"Tokenized 'ingest': {tokenized_a}")
print(f"Tokenized 'sting': {tokenized_b}")
print(f"Tokenized 'interest': {tokenized_c}")

Tokenized 'ingest': ing e st 
Tokenized 'sting': st ing 
Tokenized 'interest': in t e r e st 
